In [1]:
spark

In [2]:
sc

In [3]:
sqlContext

<h4> Read in Loan data </h4>

In [5]:
loan_df = sqlContext.read.format("csv").options(header = "true", inferschema = "true").load("/FileStore/tables/7oewn9wr1506610007136/lending_club_data-fbee9.csv")

In [6]:
display(loan_df)

<h4> Exploring Features </h4>

In [8]:
import pandas as pd
#loan_pd_df = loan_df.toPandas()


In [9]:
display(loan_df.describe())

In [10]:
loan_df.columns

In [11]:
loan_df.printSchema()

In [12]:
loan_df.cache()

<h4> Look at distribution of grade data </h4>

In [14]:
#import matplotlib as plt
#%matplotlib inline
#grade_distribution = loan_df.['grade'].value_counts()
grade_distribution = loan_df.groupBy('grade').count()
display(grade_distribution)

In [15]:
loan_df.createOrReplaceTempView('loan_tab')

<h4> Over half the loan grades are 'B or 'C' </h4>

In [17]:
display(spark.sql("select grade, count(*) as count from loan_tab group by grade order by count(*) desc"))

<h4> Only a Small percentage of loanees own a home </h4>

In [19]:
display(loan_df.groupBy('home_ownership').count().sort(desc("count")))

In [20]:
from pyspark.sql import functions as f

display(loan_df.groupBy('home_ownership').agg(f.count('*').alias('total')).sort(desc('total')))

<h4> Exploring Target Column 'Bad Loans' </h4>

<h5> Remapping it to be +1 and -1 so it is more intuitive. 
<p> 
<ul> 
<li> +1 -> safe </li>
<li> -1 -> risky (bad loan) </li>
</ul>
</p>

In [23]:
loan_df = loan_df.withColumn('bad_loans', loan_df('bad_loans').map(lambda x : +1 if x==0 else -1))
#loans = loans.remove_column('bad_loans')